In [145]:
from sklearn.feature_selection import SelectKBest
import pandas as pd
import numpy as np
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, TunedThresholdClassifierCV, FixedThresholdClassifier
import joblib

rng = np.random.RandomState(0)

In [146]:
uromol_filtered = pd.read_csv("../data/uromol_top500.csv", index_col = 0)
labels_str = pd.read_csv("../data/uromol_meta_labelled.csv", index_col = 0)['cluster'].tolist()

In [147]:
X_train, X_test, y_train, y_test = train_test_split(uromol_filtered, labels_str, test_size=0.2, random_state=rng)

pipe = Pipeline([
    ("scaler", QuantileTransformer(random_state=rng)),
    ("enlr", LogisticRegression(
        solver = "saga",
        max_iter = 5000,
        class_weight = "balanced",
        random_state = None
    ))
])

In [148]:
param_grid = {
    "enlr__l1_ratio": np.linspace(0.1, 0.9, 5),
    "enlr__C": np.logspace(-2, 2, 5),
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rng)
gs = GridSearchCV(pipe, param_grid = param_grid, scoring = 'roc_auc',
                  cv = cv, n_jobs = 4, return_train_score=True)
gs.fit(X_train, y_train)

/Users/dingma/Desktop/bc-risk-classifier/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_data.py:2885: UserWarning: n_quantiles (1000) is greater than the total number of samples (182). n_quantiles is set to n_samples.
  warnings.warn(
/Users/dingma/Desktop/bc-risk-classifier/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_data.py:2885: UserWarning: n_quantiles (1000) is greater than the total number of samples (182). n_quantiles is set to n_samples.
  warnings.warn(
/Users/dingma/Desktop/bc-risk-classifier/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_data.py:2885: UserWarning: n_quantiles (1000) is greater than the total number of samples (183). n_quantiles is set to n_samples.
  warnings.warn(
/Users/dingma/Desktop/bc-risk-classifier/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_data.py:2885: UserWarning: n_quantiles (1000) is greater than the total number of samples (182). n_quantiles is set to n_samples.
  warnings.warn(
/Users/dingma/De

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...ver='saga'))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'enlr__C': array([1.e-02...e+01, 1.e+02]), 'enlr__l1_ratio': array([0.1, 0....5, 0.7, 0.9])}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",4
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",StratifiedKFo... shuffle=True)
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parame

In [149]:
pd.DataFrame(gs.cv_results_)[
    [
        "mean_test_score",
        "param_enlr__l1_ratio",
        "param_enlr__C",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index().T

rank_test_score,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,22,22,22
mean_test_score,0.964869,0.964508,0.964119,0.963714,0.963692,0.963303,0.963222,0.962934,0.961715,0.961687,...,0.959456,0.959334,0.958093,0.954716,0.941673,0.924042,0.500000,0.500000,0.500000,0.500000
param_enlr__l1_ratio,0.900000,0.500000,0.100000,0.300000,0.700000,0.100000,0.100000,0.100000,0.300000,0.300000,...,0.500000,0.700000,0.900000,0.700000,0.100000,0.900000,0.900000,0.700000,0.500000,0.300000
param_enlr__C,100.000000,100.000000,10.000000,100.000000,100.000000,1.000000,0.100000,100.000000,1.000000,10.000000,...,0.100000,10.000000,10.000000,0.100000,0.010000,0.100000,0.010000,0.010000,0.010000,0.010000
mean_fit_time,0.915449,0.803671,0.398661,0.770147,0.865054,0.203832,0.186091,0.684578,0.272006,0.632370,...,0.149221,1.274731,1.757005,0.119954,0.082762,0.136404,0.036998,0.036418,0.041242,0.041954


In [150]:
gs.score(X_test, y_test)

0.984375

In [151]:
enlr = gs.best_estimator_.named_steps["enlr"]
coef = enlr.coef_.ravel()
n_nonzero = (coef != 0).sum()
n_total = coef.size

print(f"None-zero coefficients: {n_nonzero} / {n_total}")

None-zero coefficients: 409 / 500


In [152]:
tt = TunedThresholdClassifierCV(
    estimator=gs.best_estimator_,
    scoring="balanced_accuracy",
    cv=cv,
    n_jobs = 4,
    random_state=rng,
    store_cv_results = True
)
tt.fit(X_train, y_train)

/Users/dingma/Desktop/bc-risk-classifier/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_data.py:2885: UserWarning: n_quantiles (1000) is greater than the total number of samples (228). n_quantiles is set to n_samples.
  warnings.warn(
/Users/dingma/Desktop/bc-risk-classifier/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_data.py:2885: UserWarning: n_quantiles (1000) is greater than the total number of samples (182). n_quantiles is set to n_samples.
  warnings.warn(
/Users/dingma/Desktop/bc-risk-classifier/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_data.py:2885: UserWarning: n_quantiles (1000) is greater than the total number of samples (182). n_quantiles is set to n_samples.
  warnings.warn(
/Users/dingma/Desktop/bc-risk-classifier/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_data.py:2885: UserWarning: n_quantiles (1000) is greater than the total number of samples (182). n_quantiles is set to n_samples.
  warnings.warn(
/Users/dingma/De

,"estimator estimator: estimator instanceThe classifier, fitted or not, for which we want to optimizethe decision threshold used during `predict`.",Pipeline(step...ver='saga'))])
,"scoring scoring: str or callable, default=""balanced_accuracy""The objective metric to be optimized. Can be one of:- str: string associated to a scoring function for binary classification, see :ref:`scoring_string_names` for options.- callable: a scorer callable object (e.g., function) with signature ``scorer(estimator, X, y)``. See :ref:`scoring_callable` for details.",'balanced_accuracy'
,"response_method response_method: {""auto"", ""decision_function"", ""predict_proba""}, default=""auto""Methods by the classifier `estimator` corresponding to thedecision function for which we want to find a threshold. It can be:* if `""auto""`, it will try to invoke, for each classifier, `""predict_proba""` or `""decision_function""` in that order.* otherwise, one of `""predict_proba""` or `""decision_function""`. If the method is not implemented by the classifier, it will raise an error.",'auto'
,"thresholds thresholds: int or array-like, default=100The number of decision threshold to use when discretizing the output of theclassifier `method`. Pass an array-like to manually specify the thresholdsto use.",100
,"cv cv: int, float, cross-validation generator, iterable or ""prefit"", default=NoneDetermines the cross-validation splitting strategy to train classifier.Possible inputs for cv are:* `None`, to use the default 5-fold stratified K-fold cross validation;* An integer number, to specify the number of folds in a stratified k-fold;* A float number, to specify a single shuffle split. The floating number should be in (0, 1) and represent the size of the validation set;* An object to be used as a cross-validation generator;* An iterable yielding train, test splits;* `""prefit""`, to bypass the cross-validation.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... warning:: Using `cv=""prefit""` and passing the same dataset for fitting `estimator` and tuning the cut-off point is subject to undesired overfitting. You can refer to :ref:`TunedThresholdClassifierCV_no_cv` for an example. This option should only be used when the set used to fit `estimator` is different from the one used to tune the cut-off point (by calling :meth:`TunedThresholdClassifierCV.fit`).",StratifiedKFo... shuffle=True)
,"refit refit: bool, default=TrueWhether or not to refit the classifier on the entire training set oncethe decision threshold has been found.Note that forcing `refit=False` on cross-validation having morethan a single split will raise an error. Similarly, `refit=True` inconjunction with `cv=""prefit""` will raise an error.",True
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel. When `cv` represents across-validation strategy, the fitting and scoring on each data splitis done in parallel. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",4
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of cross-validation when `cv` is a float.See :term:`Glossary `.",RandomState(M...at 0x116A2E840
,"store_cv_results store_cv_results: bool, default=FalseWhether to store all scores and thresholds computed during the cross-validationprocess.",True
,"n_quantiles n_quantiles: int, default=1000 or n_samplesNumber of quantiles to be computed. It corresponds to the numberof landmarks used to discretize the cumulative distribution function.If n_quantiles is larger than the number of samples, n_quantiles is setto the number of samples as a larger number of quantiles does not givea better approximation of the cumulative distribution functionestimator.",1000
,"output_distribution output_distribution: {'uniform', 'normal'}, default='uniform'Marginal distribution for the transformed data. The choices are'un

In [153]:
results = (
    pd.DataFrame(tt.cv_results_).set_index("scores").sort_index(ascending=False)
)
display(results.T)

scores,0.918205,0.918205,0.918205,0.918205,0.916193,0.916193,0.916188,0.914502,0.914049,0.914049,...,0.846548,0.837921,0.832366,0.824946,0.813834,0.797180,0.780767,0.750972,0.527485,0.500000
thresholds,0.535354,0.525253,0.515152,0.505051,0.383838,0.393939,0.40404,0.494949,0.343434,0.333333,...,0.010101,0.929293,0.939394,0.949495,0.959596,0.969697,0.979798,0.989899,1.0,3.224566e-10


In [154]:
tt.score(X_test, y_test)

0.896551724137931

In [155]:
final_pipe = Pipeline([
    ("scaler", QuantileTransformer(random_state=rng)),
    ("enlr", LogisticRegression(
        solver = "saga",
        l1_ratio = gs.best_params_["enlr__l1_ratio"],
        C = gs.best_params_["enlr__C"],
        max_iter = 5000,
        class_weight="balanced",
        random_state=rng
    ))
])

final_pipe.fit(uromol_filtered, labels_str)

/Users/dingma/Desktop/bc-risk-classifier/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_data.py:2885: UserWarning: n_quantiles (1000) is greater than the total number of samples (286). n_quantiles is set to n_samples.
  warnings.warn(


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('scaler', ...), ('enlr', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"n_quantiles n_quantiles: int, default=1000 or n_samplesNumber of quantiles to be computed. It corresponds to the numberof landmarks used to discretize the cumulative distribution function.If n_quantiles is larger than the number of samples, n_quantiles is setto the number of samples as a larger number of quantiles does not givea better approximation of the cumulative distribution functionestimator.",1000
,"output_distribution output_distribution: {'uniform', 'normal'}, default='uniform'Marginal distribution for the transformed data. The choices are'uniform' (default) or 'normal'.",'uniform'
,"ignore_implicit_zeros ignore_implicit_zeros: bool, default=FalseOnly applies to sparse matrices. If True, the sparse entries of thematrix are discarded to compute the quantile statistics. If False,these entries are treated as zeros.",False
,"subsample subsample: int or None, default=10_000Maximum number of samples used to estimate the quantiles forcomputational efficiency. Note that the subsampling procedure maydiffer for value-identical sparse and dense matrices.Disable subsampling by setting `subsample=None`... versionadded:: 1.5 The option `None` to disable subsampling was added.",10000
,"random_state random_state: int, RandomState instance or None, default=NoneDetermines random number generation for subsampling and smoothingnoise.Please see ``subsample`` for more details.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",RandomState(M...at 0x116A2E840
,"copy copy: bool, default=TrueSet to False to perform inplace transformation and avoid a copy (if theinput is already a numpy array).",True
,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.

In [156]:
clf = final_pipe.named_steps["enlr"]
coef = clf.coef_.ravel()
print("None-zero genes:", (coef != 0).sum())

None-zero genes: 416


In [157]:
final_clf = FixedThresholdClassifier(
    estimator=final_pipe,
    threshold=tt.best_threshold_
)
tt.best_threshold_

np.float64(0.515151514639532)

In [158]:
joblib.dump(final_clf, "../model/final_enlr_uromol.joblib")
joblib.dump(uromol_filtered.columns.tolist(), "../model/uromol_gene_order.joblib")

['../model/uromol_gene_order.joblib']

In [159]:
knowles = pd.read_csv("../data/knowles_exprs.csv", index_col = 0)
knowles_aligned = knowles.reindex(columns=uromol_filtered.columns)

In [160]:
preds = final_clf.predict(knowles_aligned)
preds

array(['C2', 'C2', 'C1', 'C1', 'C2', 'C2', 'C2', 'C2', 'C1', 'C2', 'C2',
       'C2', 'C2', 'C2', 'C1', 'C2', 'C1', 'C1', 'C1', 'C2', 'C2', 'C2',
       'C2', 'C1', 'C2', 'C1', 'C1', 'C1', 'C2', 'C1', 'C2', 'C1', 'C1',
       'C1', 'C1', 'C2', 'C2', 'C1', 'C1', 'C2', 'C2', 'C1', 'C1', 'C1',
       'C2', 'C2', 'C1', 'C1', 'C1', 'C1', 'C2', 'C1', 'C1', 'C1', 'C1',
       'C1', 'C2', 'C2', 'C1', 'C1', 'C2', 'C2', 'C1', 'C2', 'C1', 'C1',
       'C1', 'C1', 'C1', 'C1', 'C1', 'C2', 'C2', 'C2', 'C2', 'C2', 'C1'],
      dtype='<U2')

In [161]:
prob_C2 = final_clf.predict_proba(knowles_aligned)[:, 1]
prob_C2

array([0.97420847, 0.92024463, 0.0453689 , 0.14883834, 0.96486381,
       0.67740604, 0.97755803, 0.97812523, 0.01698494, 0.66281935,
       0.97759764, 0.80007756, 0.52282162, 0.98731555, 0.00852072,
       0.98456855, 0.03760695, 0.03532715, 0.22385317, 0.99373819,
       0.96356353, 0.97845881, 0.85634257, 0.06634113, 0.92959751,
       0.01880977, 0.06162242, 0.07749761, 0.9638702 , 0.5002561 ,
       0.96241102, 0.11220174, 0.41565093, 0.04979601, 0.02672748,
       0.96354371, 0.91653941, 0.03489755, 0.01932531, 0.65491921,
       0.82249766, 0.00553307, 0.01811781, 0.13660468, 0.99217221,
       0.92994588, 0.02163804, 0.02448188, 0.13537605, 0.00766851,
       0.93177743, 0.02094482, 0.06450366, 0.01161508, 0.01599065,
       0.01507197, 0.93177153, 0.70967699, 0.00875824, 0.02181577,
       0.95267982, 0.92226802, 0.02183596, 0.63613419, 0.32822085,
       0.06501178, 0.02399687, 0.04459469, 0.5003435 , 0.00213718,
       0.0294684 , 0.95883111, 0.89028492, 0.9636641 , 0.97119

In [162]:
knowles_meta = pd.read_csv("../data/knowles_meta.csv", index_col = 0)
knowles_meta["cluster"] = preds
knowles_meta["prob_C2"] = prob_C2
knowles_meta.to_csv("../data/knowles_meta_labelled.csv", index = False)